In [17]:
import os
import subprocess
import pymysql
from bs4 import BeautifulSoup

In [18]:
path_default = "data/"
path_target = "articles.A-B.xml"
path_opennlp = "apache-opennlp-1.8.1/"

In [19]:
labels = {}
file_label = open(path_default + 'Structured-Abstracts-Labels-102615.txt', 'r')
for line in file_label.readlines() :
    label = line.split('|')
    labels[label[0]] = label[1]

In [20]:
conn = pymysql.connect(host = 'localhost', user = 'root', password = 'root', db = 'nrf')
curs = conn.cursor()

In [21]:
lines = list(map(lambda s : s.strip(), open('/home/kmh/Dataset/' + path_target + '.txt', 'r').readlines()))

In [35]:
i = 1

for path_file in lines:
    
    if i == 50 :
        break
        
    i = i + 1
    
    pmid = ''
    ppub = ''
    abstract = ''
    su = ''
    origin_label = []
    map_label = []
    s_sentences = []
    u_sentences = []
    
    try:
        article = BeautifulSoup(open('/home/kmh/Dataset/' + path_target + '/' + path_file, 'r').read(), 'html.parser')

        for node in article.select('article-id') :
            if node['pub-id-type'] == 'pmid' :
                pmid = node.text
                break

        for node in article.select('pub-date') :
            if node['pub-type'] == 'ppub' :
                ppub = '%s%02d' % (node.year.text, int(node.month.text))
                break

        for node in article.select('abstract sec') :
            origin_label.append(node.title.text.upper())
            map_label.append(labels[node.title.text.upper()])

            content = node.p.text
            abstract += content

            process_nlp = subprocess.Popen([path_opennlp + 'bin/opennlp',  'SentenceDetector', path_opennlp + 'en-sent.bin'], stdin = subprocess.PIPE, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
            stdout, stderr = process_nlp.communicate(content.encode())
            s_sentences.append(stdout.decode().split('\n'))


        su = 's' if len(s_sentences) != 0 else 'u' 
        if len(s_sentences) == 0 :
            abstract = article.select('abstract')[0].p.text

            process_nlp = subprocess.Popen([path_opennlp + 'bin/opennlp',  'SentenceDetector', path_opennlp + 'en-sent.bin'], stdin = subprocess.PIPE, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
            stdout, stderr = process_nlp.communicate(abstract.encode())
            u_sentences.append(stdout.decode().split('\n'))

        print(path_file)
        print('PMID: ' + pmid)
        print('PPUB: ' + ppub)
        print('Abstract: ' + abstract)
        print(origin_label)
        print(map_label)
        print('s/u: ' + su)

        sql_sentence = ''
        if su == 's' :
            for index in range(0, len(s_sentences) - 1, 1):
                for sentence in s_sentences[index][:-3]:
                    sen = sentence.replace("'", "")
                    sql_sentence += "insert into sentence(`pmid`, `section`, `original_section`, `sentence`) values('" + pmid + "', '" + map_label[index]  + "', '" + origin_label[index] + "', '" + sen + "'); "
        else:
            for index in range(0, len(u_sentences) - 1, 1):
                for sentence in u_sentences[index][:-3]:
                    sen = sentence.replace("'", "")
                    print('SENTENCE' + sen)
                    sql_sentence += "insert into sentence(`pmid`, `section`, `original_section`, `sentence`) values('" + pmid + "', '-', '-', '" + sen + "'); "

        sql_pmid = "insert into pmid (`pmid`, `abstract`) values('" + pmid + "', '" + abstract + "');"
        sql_abstract = "insert into abstract (`pmid`, `s/u`, `pub-date`) values('" + pmid + "', '" + su + "', '" + ppub + "');"

        sql = sql_pmid + sql_abstract + sql_sentence

        curs.execute(sql.encode())
        conn.commit()
    except:
        print('ERROR')
        print(sql)
        break


Am_J_Perinatol/PMC3966067.nxml
PMID: 21480159
PPUB: 201109
Abstract: This prospective observational study explored the association of hypertensive disorders of pregnancy and small-for-gestational age (SGA) with obstructive sleep apnea (OSA) as determined by screening measures for OSA and sleep studies.Two symptom-based screening questionnaires, the Berlin Questionnaire (BQ) and the Epworth Sleepiness Scale (ESS), were administered to enroll 1509 gravidae. Screen positive subjects were referred for polysomnography (PSG). The primary outcome was the occurrence of either gestational hypertension or preeclampsia. [a1]Generalized linear models (GLM) were used to estimate the relative risks of associations.1157 subjects were available for outcomes analysis. Screening positive on the BQ was positively associated with hypertensive disorders in GLM models (aRR=1.90, 95%CI 1.52–2.37).In this large prospective trial, GLM modeling suggest that the BQ but not the ESS demonstrated significant associ

In [9]:
conn.close()